### Model Optimizasyonu ve Overfitting Yönetimi
Selin Hoca'nın slaytındaki 'Overfitting' riskini yönetmek ve modelin genelleme yeteneğini artırmak için Optuna ile hiperparametre optimizasyonu yaptım.

Validation setini kullanarak `learning_rate`, `num_leaves` ve regülasyon (`lambda`) parametrelerini bilimsel olarak ayarladım.

In [38]:
import pandas as pd
import numpy as np
import optuna

In [39]:
import os
import joblib

In [40]:
train_df = pd.read_csv('../data/ranking/train.csv')
val_df = pd.read_csv('../data/ranking/val.csv')

In [41]:
q_train = train_df.groupby('Event_Block', sort=False).size().to_list()
q_val = val_df.groupby('Event_Block', sort=False).size().to_list()

In [53]:
drop_cols = ['Class', 'Time', 'Event_Block']

In [54]:
X_train = train_df.drop(drop_cols, axis=1)
y_train = train_df['Class']

In [55]:
X_val = val_df.drop(drop_cols, axis=1)
y_val = val_df['Class']

In [56]:
print(f"Train Gruplari: {len(q_train)} adet")
print(f"Val Gruplari: {len(q_val)} adet")

Train Gruplari: 33 adet
Val Gruplari: 7 adet


In [57]:
import lightgbm as lgb
from sklearn.metrics import ndcg_score

In [58]:
def objective(trial):
    params = {
        'objective': 'lambdarank', 
        'metric': 'ndcg',          
        'eval_at': 5,              
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
    }

    model = lgb.LGBMRanker(**params)

    model.fit(
        X_train, y_train,
        group=q_train,
        eval_set=[(X_val, y_val)],
        eval_group=[q_val],
        callbacks=[lgb.early_stopping(stopping_rounds=50)] 
    )

    score = model.best_score_['valid_0']['ndcg@5']
    return score

print("Optimizasyon başlıyor... (Bu işlem birkaç dakika sürebilir)")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print("\n------------------------------------------------")
print("EN İYİ PARAMETRELER:", study.best_params)
print("EN İYİ SKOR (NDCG@5):", study.best_value)
print("------------------------------------------------")

[I 2025-12-01 23:53:42,044] A new study created in memory with name: no-name-5563f767-0df8-4522-af4a-9c3cc8e5f257


Optimizasyon başlıyor... (Bu işlem birkaç dakika sürebilir)
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-01 23:53:42,588] Trial 0 finished with value: 0.9603895103778503 and parameters: {'learning_rate': 0.18220039063807442, 'num_leaves': 41, 'lambda_l1': 9.40875096434723, 'lambda_l2': 1.6564274910709353, 'bagging_fraction': 0.59814878239506, 'feature_fraction': 0.5499314480931168}. Best is trial 0 with value: 0.9603895103778503.


Early stopping, best iteration is:
[29]	valid_0's ndcg@5: 0.96039
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-01 23:53:43,097] Trial 1 finished with value: 0.9603895103778503 and parameters: {'learning_rate': 0.19511587706273342, 'num_leaves': 124, 'lambda_l1': 8.265091392456004, 'lambda_l2': 5.740962831374724, 'bagging_fraction': 0.5454587244276248, 'feature_fraction': 0.7385304986203407}. Best is trial 0 with value: 0.9603895103778503.


...

Did not meet early stopping. Best iteration is:
[62]	valid_0's ndcg@5: 1
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-01 23:53:55,745] Trial 18 finished with value: 0.9603895103778503 and parameters: {'learning_rate': 0.08289941943631444, 'num_leaves': 87, 'lambda_l1': 5.863973702758553, 'lambda_l2': 6.637826250447291, 'bagging_fraction': 0.591995298357778, 'feature_fraction': 0.6913010916892359}. Best is trial 4 with value: 1.0.


Early stopping, best iteration is:
[42]	valid_0's ndcg@5: 0.96039
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-12-01 23:53:56,602] Trial 19 finished with value: 1.0 and parameters: {'learning_rate': 0.15947407843120223, 'num_leaves': 22, 'lambda_l1': 1.7510221106881967, 'lambda_l2': 4.714491015151293, 'bagging_fraction': 0.7906168588629652, 'feature_fraction': 0.9519789808898135}. Best is trial 4 with value: 1.0.


Did not meet early stopping. Best iteration is:
[55]	valid_0's ndcg@5: 1

------------------------------------------------
EN İYİ PARAMETRELER: {'learning_rate': 0.0868193414184596, 'num_leaves': 26, 'lambda_l1': 0.10660088294024428, 'lambda_l2': 4.157067925555031, 'bagging_fraction': 0.5429073696880456, 'feature_fraction': 0.9326918410538783}
EN İYİ SKOR (NDCG@5): 1.0
------------------------------------------------


In [59]:
best_params = study.best_params

In [60]:
best_params['objective'] = 'lambdarank'
best_params['metric'] = 'ndcg'
best_params['eval_at'] = 5
best_params['random_state'] = 42

In [61]:
final_model = lgb.LGBMRanker(**best_params)

In [62]:
final_model.fit(
    X_train, y_train,
    group=q_train,
    eval_set=[(X_val, y_val)],
    eval_group=[q_val],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Did not meet early stopping. Best iteration is:
[57]	valid_0's ndcg@5: 1


,boosting_type,'gbdt'
,num_leaves,26
,max_depth,-1
,learning_rate,0.0868193414184596
,n_estimators,100
,subsample_for_bin,200000
,objective,'lambdarank'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [63]:
os.makedirs('../models', exist_ok=True)
joblib.dump(final_model, '../models/lgbm_ranker.pkl')

['../models/lgbm_ranker.pkl']